In [1]:
import requests
import re
import mysql.connector
import pandas as pd
mydb = mysql.connector.connect(
  host="163.44.196.192",
  user="ararize",
  password="isylzjko$3rv3r",
  database="tp"
)
mycursor = mydb.cursor()

In [5]:
def request_colData():
    sql = """SELECT bef.meID, 
            bef.bphsflow AS steam_productivity_bef, 
            bef.bphspress AS steam_pressure_bef, 
            bef.bcKghrMw AS feedwater_rate_bef, 
            bef.bcCTw AS feedwater_temp_bef, 
            bef.bphTdsInput AS TDS_feed_bef , 
            bef.bphTdsBlow AS TDS_blowdown_bef, 
            bef.bdEnBlowkg AS blowdown_bef, 
            aft.bhdsflow AS steam_productivity_aft, 
            aft.bhdspress AS steam_pressure_aft, 
            aft.bcKghrMw AS feedwater_rate_aft, 
            aft.bcCTw AS feedwater_temp_aft, 
            aft.bhdTdsInput AS TDS_feed_aft,
            aft.bhdTdsBlow AS TDS_blowdown_aft, 
            aft.bdEnBlowkg AS blowdown_aft
            FROM (SELECT bef_1.meID,bef_1.bphsflow, bef_1.bphspress, bef_2.bcKghrMw, bef_2.bcCTw, bef_1.bphTdsInput, bef_1.bphTdsBlow, bef_2.bdEnBlowkg 
                FROM`boiler_pf_history` AS bef_1 
                INNER JOIN (SELECT meID, bcKghrMw, bcCTw, bdEnBlowkg 
                            FROM `boiler_data`
                            WHERE `bdType` = 1) AS bef_2
                ON bef_1.meID = bef_2.meID) AS bef
            INNER JOIN (SELECT aft_1.meID, aft_1.bhdsflow, aft_1.bhdspress, aft_2.bcKghrMw, aft_2.bcCTw, aft_1.bhdTdsInput, aft_1.bhdTdsBlow, aft_2.bdEnBlowkg
                        FROM (SELECT * 
                            FROM `boiler_history_data` 
                            WHERE bhdType = 3) AS aft_1 
                        INNER JOIN (SELECT meID, bcKghrMw, bcCTw, bdEnBlowkg 
                            FROM `boiler_data`
                            WHERE `bdType` = 3) AS aft_2
                ON aft_1.meID = aft_2.meID) AS aft
            ON bef.meID = aft.meID"""
    mycursor.execute(sql)
    myresult = mycursor.fetchall()
    colName = ['meID',
            'steam_productivity_bef',
            'steam_pressure_bef',
            'feedwater_rate_bef',
            'feedwater_temp_bef',
            'TDS_feed_bef',
            'TDS_blowdown_bef',
            'blowdown_bef',
            'steam_productivity_aft',
            'steam_pressure_aft',
            'feedwater_rate_aft',
            'feedwater_temp_aft',
            'TDS_feed_aft',
            'TDS_blowdown_aft',
            'blowdown_aft']
    col_data = pd.DataFrame(myresult, columns = colName)
    col_data['meID'] = col_data['meID'].astype('str')
    return col_data

def request_calBoilerAPI():
    url = 'http://tp.besten.co.th/backend/web/index.php?r=report/json-me&user=admin&pass=123456'
    df_json = pd.read_json(url)
    data = pd.DataFrame(df_json.iloc[0,0],index=[0])
    for i in range(len(df_json)):
        data = data.append(pd.DataFrame(df_json.iloc[i,0],index=[i]))
    cal_data = data.drop_duplicates()
    cal_data['meID'] = cal_data['meID'].astype('str')
    return cal_data

def findCol(colName_list,target):
    a=[]
    for i in colName_list:
        for j in target:
            if j in i :
                a.append(i)
                break
    return a
def request_BoilerID():
    sql = "SELECT * FROM `boilerID`"
    mycursor.execute(sql)
    myresult = mycursor.fetchall()
    boilerID_data = pd.DataFrame(myresult, columns = ['meID','BoilerID'])
    boilerID_data['meID'] = boilerID_data['meID'].astype('str')
    return boilerID_data

col_data = request_colData()
cal_data = request_calBoilerAPI()
select_col = findCol(cal_data.columns,['meID','mf_','hhvdb_','HV_','_o2','_tg_','_co_','toe','save','inv','s2_','s8_','s9_'])
cal_data = cal_data[select_col]
df_whole = col_data.merge(cal_data,how='inner',on='meID')
boilerID_data = request_BoilerID()
df_whole = boilerID_data.merge(df_whole,how='left',on='meID')

#---------------------------------------------------------------------------------------------#
df_info = pd.read_csv('total_data.csv')
df_info = df_info.iloc[:,0:8]
df_whole = df_info.merge(df_whole,how='left',on='BoilerID')
df_whole.drop('meID',axis=1,inplace=True)

#---------------------------------------------------------------------------------------------#
phrase = ['bef','pot','aft']
for i in phrase:
    a = []
    search_txt = 'L.+HV.+'+i
    for j in df_whole.columns:
        if re.search(search_txt,j):
            a.append(j)
    for k in a:
        df_whole[k] = df_whole[k].astype('float')
    col_eff_name = 'IndirectEFF_LHV_'+i+'_[%]'
    df_whole[col_eff_name] = 100
    for j in a[0:6]:
        df_whole[col_eff_name] = df_whole[col_eff_name]-df_whole[j]
    col_eff_name = 'IndirectEFF_HHV_'+i+'_[%]'
    df_whole[col_eff_name] = 100
    for j in a[6:]:
        df_whole[col_eff_name] = df_whole[col_eff_name]-df_whole[j]


df_whole.rename(columns={'mf_bef':'FuelFlowRate_bef[kg/hr]',
                        'mf_pot':'FuelFlowRate_pot[kg/hr]',
                        'mf_aft':'FuelFlowRate_aft[kg/hr]',
                        'hhvdb_bef':'HHV_DB_bef[kJ/kg]',
                        'hhvdb_pot':'HHV_DB_pot[kJ/kg]',
                        'hhvdb_aft':'HHV_DB_aft[kJ/kg]'},inplace=True)


In [6]:
df_whole.head()

,BoilerID,BoilerType,Capacity,ImprovementTech,LifeTime,FuelName,FuelType,Burner,steam_productivity_bef,steam_pressure_bef,...,s9_qtstoe_aft,s9_cts_aft,s9_c_aft,s9_pb_aft,IndirectEFF_LHV_bef_[%],IndirectEFF_HHV_bef_[%],IndirectEFF_LHV_pot_[%],IndirectEFF_HHV_pot_[%],IndirectEFF_LHV_aft_[%],IndirectEFF_HHV_aft_[%]
0,10621-0019-1,water tube,4.0,NaN,10.0,แกลบ,Solid,overfeed stoker (Fixed/Inclined Grate) - Aut...,None,14.51,...,0.00,0.00,0.00,0.00,83.87,76.32,86.78,79.06,84.91,77.29
1,10132-0005-1,fire tube,3.0,"Economizer, นำคอนเดนเสทกลับมาใช้",15.0,LPG,Gas,แบบผสมกันภายในหัวเผา (Nozzle-mix burner),892.06,9.50,...,0.00,0.00,0.00,0.00,84.99,78.05,88.80,81.55,86.73,79.65
2,10120-0017-1,fire tube,12.0,นำคอนเดนเสทกลับมาใช้,19.0,LPG,Gas,แบบผสมเชื้อเพลิงกับอากาศก่อนป้อนเข้าเผาไหม้ (...,3600.00,8.30,...,488.59,"7,000,000.00",0.00,0.00,88.29,81.08,90.83,83.43,88.29,81.08
3,30911-0005-1,once through,1.0,นำคอนเดนเสทกลับมาใช้,11.0,LPG,Gas,แบบผสมเชื้อเพลิงกับอากาศก่อนป้อนเข้าเผาไหม้ (...,341.41,7.01,...,0.00,0.00,0.00,0.00,90.23,82.87,93.19,85.59,90.12,82.78
4,13132-1001-1,fire tube,12.0,นำคอนเดนเสทกลับมาใช้,20.0,กะลาปาล์ม,Solid,overfeed stoker (Traveling Grate) - Auto Feed,2690.24,8.50,...,0.00,0.00,0.00,0.00,75.03,65.45,83.31,73.24,78.21,68.45


In [7]:
df_whole.drop(['me_HLHV_bef','me_HHHV_bef'],axis=1).set_index('BoilerID').to_csv('boiler_Data_new.csv')

#### Ajan Ja #####

In [ ]:

import math

def calBlow(Feed,Blown,TDS_Feed,TDS_Blown):
    #print(Feed,Blown,TDS_Feed,TDS_Blown)
    if math.isnan(Feed):
        return Blown
    else :
        if Feed>0:
            return TDS_Feed/TDS_Blown*Feed
        else:
            return Blown

def calSteam(Steam,Feed,Blown):
    #print(Steam,Feed,Blown)
    if math.isnan(Steam):
        return Feed-Blown
    else :
        if Steam > 0:
            return Steam
        else :
            return Feed-Blown


In [ ]:
df_ja.to_csv('DataSupportAjarnJA.csv')

In [ ]:
def request_calSysAPI():
    url = 'http://tp.besten.co.th/backend/web/index.php?r=report/json-mc&user=admin&pass=123456'
    df_json = pd.read_json(url)
    data = pd.DataFrame(df_json.iloc[0,0],index=[0])
    for i in range(len(df_json)):
        data = data.append(pd.DataFrame(df_json.iloc[i,0],index=[i]))
    cal_data = data.drop_duplicates()
    cal_data['mcID'] = cal_data['mcID'].astype('str')
    return cal_data
def request_TsicID():
    sql = 'SELECT mcID, mcTsic, mcCompName FROM `maincomp`'
    mycursor.execute(sql)
    myresult = mycursor.fetchall()
    colName = ['mcID','TSIC','CompanyName']
    TsicID_data = pd.DataFrame(myresult, columns = colName)
    TsicID_data['mcID'] = TsicID_data['mcID'].astype('str')
    return TsicID_data

TsicID = request_TsicID()
calSys_data = request_calSysAPI()
df_sys = TsicID.merge(calSys_data,how='left',on='mcID')
#df_sys.head()
df_sys.drop(0).drop('mcID',axis=1).set_index('TSIC').to_csv('system_Data.csv')

In [82]:
def map_code(map_series,map_list):
    for i in range(len(map_list)):
        map_series = map_series.str.replace(map_list[i][0],map_list[i][1])
    return map_series

def request_improTech():
    sql = 'SELECT meID, mepPerf FROM `main_equipment_perf`'
    mycursor.execute(sql)
    myresult = mycursor.fetchall()
    myresult = pd.DataFrame(myresult, columns=['meID','improTech'])
    for i in myresult.columns:
        myresult[i] = myresult[i].astype('str')
    df_improTech = myresult
    df_improTech = df_improTech.sort_values(by=['meID','improTech']).groupby('meID').sum()
    map_tech = [['1','OilHeater,'],['2','AirPreHeater,'],['3','Economizer,'],['4','CondensateReturn,']]
    df_improTech['improTech'] = map_code(df_improTech['improTech'],map_tech)
    return df_improTech.reset_index()

def request_fuelType():
    sql = 'SELECT meID, mefFuel FROM `main_equipment_fuel`'
    mycursor.execute(sql)
    myresult = mycursor.fetchall()
    myresult = pd.DataFrame(myresult, columns=['meID','fuelType'])
    for i in myresult.columns:
        myresult[i] = myresult[i].astype('str')
    df_fuelType = myresult
    map_fuel = [['1','Gas'],['2','Solid'],['3','Liquid']]
    df_fuelType['fuelType'] = map_code(df_fuelType['fuelType'],map_fuel)
    return df_fuelType

def request_burnerTech(meID,fuelType):
    #SELECT `meID`,`meBurnerType`,`meBurnerFluid`,`meControl`,`meBurnerGas`,`meControlGas` FROM `main_equipment`
    if 'Gas' in fuelType:
        sql = 'SELECT `meBurnerGas` FROM `main_equipment` WHERE `meID` = '
    elif 'Solid' in fuelType:
        sql = 'SELECT `meBurnerType` FROM `main_equipment` WHERE `meID` = '
    elif 'Liquid' in fuelType:
        sql = 'SELECT `meBurnerFluid` FROM `main_equipment` WHERE `meID` = '
    
    sql = sql + str(meID)
    mycursor.execute(sql)
    myresult = mycursor.fetchall()
    myresult = pd.DataFrame(myresult)
    
    return myresult.iloc[0,0]

def 

#request_improTech()
df_burnerTech = request_fuelType()
df_burnerTech['burner'] = ""
for i in range(len(df_burnerTech)):
    #print(request_burnerTech(meID=df_burnerTech.iloc[i,0],fuelType=df_burnerTech.iloc[i,1]))
    df_burnerTech.iloc[i,2] = request_burnerTech(meID=df_burnerTech.iloc[i,0],fuelType=df_burnerTech.iloc[i,1])
df_burnerTech['burner'] = df_burnerTech['burner'].astype('str')
df_burnerTech['meID'] = df_burnerTech['meID'].astype('str')
df_burnerTech.loc[df_burnerTech['fuelType']=='Gas','burner'] = map_code(map_series=df_burnerTech.loc[df_burnerTech['fuelType']=='Gas','burner'],map_list=[['1','Pre-mix'],['2','Nozzle-mix']])
df_burnerTech

,meID,fuelType,burner
0,19,Solid,4
1,30,Liquid,1
2,31,Liquid,1
3,45,Liquid,1
4,46,Gas,Nozzle-mix
...,...,...,...
123,83,Solid,2
124,128,Liquid,2
125,129,Liquid,2
126,20,Liquid,1
